In [9]:
import numpy as np
import pandas as pd

import gensim
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
from transformers import TFDistilBertModel, DistilBertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense
from keras.metrics import Precision, Recall
from keras.utils import to_categorical

import warnings
warnings.filterwarnings("ignore")


In [10]:
# Загрузка данных из CSV файла
df = pd.read_csv('./combined_data.csv')
df

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...
...,...,...
83443,0,hi given a date how do i get the last date of ...
83444,1,now you can order software on cd or download i...
83445,1,dear valued member canadianpharmacy provides a...
83446,0,subscribe change profile contact us long term ...


In [11]:
# Загрузка стоп-слов для английского языка из библиотеки NLTK
nltk.download('stopwords')

# Добавление дополнительных стоп-слов 
stop_words = stopwords.words('english')
stop_words.extend(['escapenumber'])

# Функция для предобработки текста
def preprocess(text, join_back=True):
    result = []
    # Применение simple_preprocess из библиотеки gensim для токенизации текста
    for token in gensim.utils.simple_preprocess(text):
        # Очистка от стоп-слов
        if (
            token not in gensim.parsing.preprocessing.STOPWORDS and 
            token not in stop_words
        ):
            result.append(token)
    # Объединение токенов обратно в строку
    if join_back:
        result = " ".join(result)
    return result


# Применение функции preprocess к столбцу 'text' и создание нового столбца 'clean_text'
df['clean_text']=df['text'].apply(preprocess)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\apple\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# Разделение данных на обучающий и тестовый наборы
train, test = train_test_split(df, test_size=0.2)

# Инициализация токенизатора с ограничением числа слов до 5000
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train['clean_text'])

# Преобразование текста в последовательности чисел с использованием токенизатора
X_train = tokenizer.texts_to_sequences(train['clean_text'])
X_test = tokenizer.texts_to_sequences(test['clean_text'])

# Выравнивание последовательностей до фиксированной длины (maxlen=100)
X_train = pad_sequences(X_train, maxlen=100)
X_test = pad_sequences(X_test, maxlen=100)

# Преобразование меток в формат one-hot encoding
Y_train = to_categorical(train['label'], num_classes=2)
Y_test = to_categorical(test['label'], num_classes=2)

loss_ = 'categorical_crossentropy'
optimizer_ = 'adam'

model = Sequential()
model.add(Embedding(8000, 100, input_length=100))
model.add(GRU(128))
model.add(Dense(2, activation='softmax'))

model.compile(loss=loss_, optimizer=optimizer_, metrics=[Precision(), Recall(), 'accuracy'])

model.fit(X_train, Y_train, 
          validation_data=(X_test, Y_test),
          epochs=3, batch_size=32)

Epoch 1/3
2087/2087 [==============================] - 70s 33ms/step - loss: 0.0769 - precision: 0.9731 - recall: 0.9731 - accuracy: 0.9731 - val_loss: 0.0484 - val_precision: 0.9841 - val_recall: 0.9841 - val_accuracy: 0.9841
Epoch 2/3
2087/2087 [==============================] - 67s 32ms/step - loss: 0.0327 - precision: 0.9895 - recall: 0.9895 - accuracy: 0.9895 - val_loss: 0.0520 - val_precision: 0.9830 - val_recall: 0.9830 - val_accuracy: 0.9830
Epoch 3/3
2087/2087 [==============================] - 67s 32ms/step - loss: 0.0185 - precision: 0.9941 - recall: 0.9941 - accuracy: 0.9941 - val_loss: 0.0596 - val_precision: 0.9832 - val_recall: 0.9832 - val_accuracy: 0.9832
